In [2]:
import pandas as pd
from g4f.client import Client
import re
client = Client()

In [3]:
def load_data(file_path: str) -> pd.DataFrame:
    """
    加載 CSV 文件並處理錯誤行。
    """
    try:
        return pd.read_csv(file_path, on_bad_lines="skip", encoding="utf-8")
    except pd.errors.ParserError as e:
        raise Exception(f"CSV 文件解析錯誤：{e}")
    except Exception as e:
        raise Exception(f"讀取數據時發生錯誤：{e}")



In [4]:
def generate_summary_statistics(data):
    """
    生成數據的描述性統計量。

    Args:
        data (DataFrame): 輸入的數據。

    Returns:
        DataFrame: 描述性統計量。
    """
    return data.describe(include='all')



In [5]:
def construct_prompt(user_input, summary_stats):
    """
    根據用戶輸入與數據統計生成提示。

    Args:
        user_input (str): 用戶的查詢。
        summary_stats (DataFrame): 數據的描述性統計量。

    Returns:
        tuple: 系統提示與用戶提示。
    """
    system_prompt = (
        "你是一位專業的數據科學家，擅長分析數據並從中提取見解。"
        "以下是一些描述性統計數據，請根據這些數據提供深入分析和建議：\n"
        "1. 描述數據的整體特性，例如平均值、標準差、數據分佈等。\n"
        "2. 尋找可能的異常值或數據問題，並評估其對分析結果的影響。\n"
        "3. 如果有類別型數據，請分析其分佈特性和可能的相關性。\n"
        "4. 根據數據，提供有助於業務決策的具體建議，例如潛在的改進方向或應關注的特定領域。\n"
        "5. 將您的分析結論簡潔清楚地表達，並建議下一步的數據收集或分析方向。")
    


    user_prompt = (
        f"數據的描述性統計如下：\n{summary_stats}\n"
        f"請根據這些數據回答以下問題：{user_input}"
    )

    "請分析以下股票數據，並給出專業的分析報告"

    return system_prompt, user_prompt


In [6]:
def generate_report(user_input, csv_file_path):
    """
    基於自然語言查詢和CSV數據生成分析報告。

    Args:
        user_input (str): 用戶查詢。
        csv_file_path (str): CSV文件路徑。

    Returns:
        str: 分析報告或錯誤信息。
    """
    data = load_data(csv_file_path)
    if isinstance(data, str):  # 返回錯誤信息時
        return data
    summary_stats = generate_summary_statistics(data)
    system_prompt, user_prompt = construct_prompt(user_input, summary_stats)
    client = Client()
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"生成報告時發生錯誤：{e}"


In [8]:
if __name__ == "__main__":
    csv_file = r".\csv.ingredient\tsmc_operating_profit_margin_2010_2023.csv"
    user_query = "請分析數據並提供詳細見解和建議。"

    report = generate_report(user_query, csv_file)
    print("生成的報告：")
    print(report)



{'chatId': 'ff650f83-b6fe-415a-9deb-32f58ece453c', 'frequency_penalty': 0, 'max_tokens': 4000, 'messages': [{'role': 'system', 'content': '你是一位專業的數據科學家，擅長分析數據並從中提取見解。以下是一些描述性統計數據，請根據這些數據提供深入分析和建議：\n1. 描述數據的整體特性，例如平均值、標準差、數據分佈等。\n2. 尋找可能的異常值或數據問題，並評估其對分析結果的影響。\n3. 如果有類別型數據，請分析其分佈特性和可能的相關性。\n4. 根據數據，提供有助於業務決策的具體建議，例如潛在的改進方向或應關注的特定領域。\n5. 將您的分析結論簡潔清楚地表達，並建議下一步的數據收集或分析方向。'}, {'role': 'user', 'content': '數據的描述性統計如下：\n              Year Quarter  Operating Profit Margin (%)\ncount     56.00000      56                    56.000000\nunique         NaN       4                          NaN\ntop            NaN      Q1                          NaN\nfreq           NaN      14                          NaN\nmean    2016.50000     NaN                    38.383929\nstd        4.06761     NaN                     2.332850\nmin     2010.00000     NaN                    33.600000\n25%     2013.00000     NaN                    36.385000\n50%     2016.50000     NaN                    38.625000\n75%     2020.0

c:\Users\TMP214\AppData\Local\Programs\Python\Python310\lib\asyncio\events.py:80: UserWarning: Curlm alread closed! quitting from process_data
  self._context.run(self._callback, *self._args)


生成的報告：
根據提供的描述性統計數據，我們可以進行以下分析和建議：

### 1. 數據的整體特性
- **平均值**：操作利潤率的平均值為38.38%，這表明在觀察期間內，公司的利潤表現相對穩定。
- **標準差**：標準差為2.33%，顯示出利潤率的變異性相對較小，數據分佈較為集中。
- **數據分佈**：
  - 最小值為33.60%，最大值為42.81%，顯示出利潤率在這段時間內有一定的波動。
  - 四分位數顯示，25%的數據低於36.39%，50%的數據（中位數）為38.63%，75%的數據低於40.09%。這表明大多數數據集中在38%到40%之間。

### 2. 異常值或數據問題
- 在這組數據中，沒有明顯的異常值（outliers），因為所有的利潤率都在33.60%到42.81%之間，且標準差相對較小。
- 然而，應注意數據的完整性和準確性，特別是在不同季度的數據是否均衡收集，這可能影響分析結果。

### 3. 類別型數據分析
- **季度分佈**：數據顯示有4個季度（Q1, Q2, Q3, Q4），其中Q1的出現頻率最高（14次），這可能表明公司在第一季度的業務表現較為穩定或有季節性影響。
- 建議進一步分析不同季度的利潤率變化，以了解季節性因素對業務的影響。

### 4. 業務決策建議
- **潛在改進方向**：考慮在利潤率較低的年份（如2010年）進行深入分析，找出原因並制定改進計劃。
- **關注特定領域**：建議關注Q1的業務表現，因為其利潤率較高，並探索是否可以在其他季度複製這一成功模式。
- **持續監測**：定期更新和監測利潤率數據，以便及時調整業務策略。

### 5. 下一步的數據收集或分析方向
- **數據收集**：建議收集更長時間範圍內的數據，以便進行時間序列分析，了解利潤率的長期趨勢。
- **深入分析**：進一步分析不同季度的利潤率變化，並考慮其他可能影響利潤率的因素（如市場環境、競爭狀況等）。

總結來說，這組數據顯示出公司在觀察期間內的利潤率相對穩定，建議進一步分析季度間的差異及其影響因素，以便制定更具針對性的業務策略。
